This script reproduces what the main.py function does but divided into its parts so we can visualize the whole process

In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import argparse
from ucimlrepo import fetch_ucirepo 

In [ ]:
from src.dataset import Data_handling
from utils.datasets_generation import generate_dataset
from src.weakener import Weakener
from src.model import MLP

In [ ]:
reps = 10
dataset_base_path = 'Datasets/weak_datasets'
dataset = 'image'
corruption = 'pll'
corr_p = 0.5
corr_n = None

for i in range(reps):
        generate_dataset(dataset=dataset,corruption=corruption,corr_p=corr_p,repetitions=i)


In [ ]:
for i in range(r)